# **Phases 1 and 2 Revisited**

## *Prompt*

**Microsoft's Making Moves into Movies**

>* What types of movies are performing the best a the box offices?
>* What actions should they take based on the data?

## *Questions*

**What questions will guide my process?**

>* What release times are best for gross value?
>* Which directors show the strongest/weakest performance?
>* What features would give the strongest indications of performance?
>* How can we determine whether or not a movie is "successful" or not?

**What preconceived ideas do I have about the data? What guesses can I make now?**

>* 
>* 

## *Process*

>1. Import .csv's
>2. Clean data
>3. EDA w. visuals
>4. **Determine initial insights and actions**
>5. Create new features
>6. Test for correlations/multicollinearity
>7. Perform statistical testing
>8. Create LinReg model for **inference**
>9. Create LinReg model for **predictions**
>10. **Present final results for inferences, predictions**
    1. Include initial, final insights and recommendations

# **Imports**

## Packages

In [ ]:
## Accessing stored data
import csv
import os,glob

## Data exploration and statistics
import pandas as pd
import numpy as np
from sklearn import preprocessing

## Creating Visualizations
import seaborn as sns
import matplotlib.pyplot as plt

## Settings
%matplotlib inline
pd.set_option('display.float_format', lambda x: '%.2f' % x)
sns.set_context('notebook', font_scale=1.25)

## Data

In [ ]:
## Creating list of files to loop through for data
data_folder = 'zippedData/'
data_files = glob.glob(f'{data_folder}*.csv*')
data_files

In [ ]:
## Looping through individual data files

## Code adapted from James Irving
## Source: youtube.com/watch?v=rufvTgBEYN8&list=PLFknVelSJiSxSwXifV_ysDg50fzbuTzVt&index=41

clean_file_names = {}
split = '-----'*25

for file in data_files:
    name = file.replace('.csv.gz','').split('\\')[-1].replace('.','_')
    print(split)
    
    print(f"Preview of {name}:")
    clean_file_names[name] = pd.read_csv(file)
    display(clean_file_names[name].head(5))
    print()

# **Functions**

In [ ]:
## Not working - unknown error

## Data Type Conversion

# def split_str(df, list_col, change, result):
#     '''Splits strings in a list of columns based on what value to change 
#     and the desired result.
    
#     Args:
#         * dataframe source
#         * list of selected columns
#         * charater to change from (str)
#         * charater to change to (str)'''
    
    
#     for i in list_col:
#         try:
#             df[i] = df[i].map((lambda x: int(x.replace(change,result))))

#         except Exception:
#             print('---'*25)
#             print(f'Already converted {df[i]}')
#             print()

# #         display(df[i])

#         return df

# **Data Cleaning**

## Slicing desired DataFrames from Dictionary for Exploration

In [ ]:
## Selecting for genre details
title_basics = clean_file_names['imdb_title_basics']
title_basics

In [ ]:
## Selecting for budget and gross details
movie_basics = clean_file_names['tn_movie_budgets']
movie_basics

In [ ]:
## Selecting for gross details
movie_gross = clean_file_names['bom_movie_gross']
movie_gross

## Merging Dataframes

In [ ]:
## Merging title_basics and movie_basics on primary title

merged_basics_primary = pd.merge(title_basics,movie_basics, 
                                 left_on= 'primary_title', right_on= 'movie')
merged_basics_primary

In [ ]:
# ## Deprecated - focused on larger data set - Merging title_basics and movie_basics on original title

# merged_basics_original = pd.merge(title_basics,movie_basics, 
#                                   left_on= 'original_title', right_on= 'movie')
# merged_basics_original

In [ ]:
df = merged_basics_primary.copy()

## Removing Redundant "Movie" Column

In [ ]:
## Verifying all titles/movies match
# df[df.loc[:,'primary_title'] != df.loc[:,'movie']]

In [ ]:
## Verifying all titles/movies match - np.where faster than logical slicing 
for x in np.where((df.loc[:,'primary_title'] != df.loc[:,'movie']),1,0):
    if x == 1:
        print(x)

In [ ]:
## Dropping redundant "movie" column -  incl t/e to be able to rerun notebook

for col in df.columns:
    try:
        df.drop('movie', axis=1, inplace=True)
    except:
        pass
    
## Confirming removal
'movie' in df.columns

## Converting Currencies from Str to Int

In [ ]:
## Converting gross amounts from strings to integers and removing 

dollar_to_int = ['production_budget','worldwide_gross','domestic_gross']

for i in dollar_to_int:
    try:
        df[i] = df[i].map((lambda x: int(x.replace('$','').replace(',',''))))

    except Exception:
        print('---'*25)
        print('Already converted.')
        print()
        
    display(df[i][:3])

## Filling Null Values with "Missing"

In [ ]:
## Filling null values with "missing" to allow further processing

df_filled = df.fillna('Missing').copy()
display(df.isna().sum())
display(df_filled.isna().sum())

# **Feature Engineering**

## Creating Seasons and Quarters

### Datetime Approach

In [ ]:
## Changing release date to datetime datatype

df_filled['release_datetime'] = pd.to_datetime(df_filled['release_date'])
df_filled['release_datetime']

In [ ]:
## Using datetime dtype to create months column
df_filled['release_month_dt'] = df_filled['release_datetime'].dt.month_name()
df_filled['release_month_dt']

In [ ]:
df_filled['release_quarter'] = df_filled['release_datetime'].dt.quarter
df_filled['release_quarter']

### Manual approach

In [ ]:
## Inspecting original values
df_filled['release_date'][:5]

In [ ]:
## Pulling month
test_month = df_filled['release_date'][0][:3]
test_month

In [ ]:
## Creating new column for the month of each release date
release_month = []

for movie in df_filled['release_date']:
    release_month.append(movie[:3])
    
df_filled['release_month_manual'] = release_month

In [ ]:
## Using map and lambda functions to slice out month from string
df_filled['release_month_manual'] = df_filled['release_date'].map(lambda x: x[:3])
df_filled['release_month_manual']

In [ ]:
## Creating seasons based on meteorological definitions of each season
season = []

for month in df_filled['release_month_manual']:
    if month == 'Jan':
        season.append('Winter')
    elif month == 'Feb':
        season.append('Winter')
    elif month == 'Mar':
        season.append('Spring')
    elif month == 'Apr':
        season.append('Spring')
    elif month == 'May':
        season.append('Spring')
    elif month == 'Jun':
        season.append('Summer')
    elif month == 'Jul':
        season.append('Summer')
    elif month == 'Aug':
        season.append('Summer')
    elif month == 'Sep':
        season.append('Fall')
    elif month == 'Oct':
        season.append('Fall')
    elif month == 'Nov':
        season.append('Fall')
    elif month == 'Dec':
        season.append('Winter')
    else:
        print('na')

df_filled['release_season_manual'] = season
df_filled['release_season_manual']

### Reviewing Changes

In [ ]:
df_filled

## Splitting Genres into Lists

In [ ]:
# ## Via map & lambda - slower than .str
# df_filled['genres_list'] = df_filled['genres'].map(lambda x: x.split(','))
# df_filled

In [ ]:
## Via .str and string methods (faster than map/lambda)
df_filled['genres_str'] = df_filled['genres'].str.title().str.split(',')
df_filled['genres_str']

In [ ]:
df_filled

## Creating Profit and ROI

In [ ]:
df_filled['profit'] = df_filled['worldwide_gross'] - df_filled['production_budget']
df_filled['profit'] 

In [ ]:
df_filled['ROI'] = (df_filled['worldwide_gross'] - df_filled['production_budget'])/df_filled['production_budget']
df_filled['ROI']

# **Visualizations**

In [ ]:
## Creating new rows for each genre per movie
plot_df = df_filled.explode('genres_str')
plot_df

## Worldwide Gross per Genre

In [ ]:
## Visualizing results

plt.figure(figsize=(15,4))
sns.barplot(data=plot_df, x= 'genres_str', y='worldwide_gross')
plt.xticks(rotation=45, ha= 'right')
plt.suptitle('Worldwide Gross by Genre')
plt.xlabel('Genres')
plt.ylabel('Worldwide Gross ($)');

**Observations:**
>* **Top three genres:** Animation, Adventure, and Sci-Fi
>* **Lowest three genres:** Reality-TV, War and News
>* *Musicals are a high-risk, high-reward option*
    * Their gross can exceed Animation, or fall below the top 5 genres.

**Suggestions**
>* **Safest Genres (by Gross)** are the top three genres
    * Lowest points on the error bars indicate high gross performance even at their worst
>* Select from Action, Animation, Adventure, Fantasy, Sci-Fi, Family, or Musicals
    * All others show poor performance

## ROI per Genre

In [ ]:
mean_roi = list(plot_df.groupby("genres_str").mean()['ROI'].sort_values(ascending=False).index)
mean_roi

In [ ]:
## Visualizing results

plt.figure(figsize=(15,4))
sns.barplot(data=plot_df, x= 'genres_str', y='ROI', order=mean_roi)
plt.xticks(rotation=45, ha= 'right')
plt.suptitle('Worldwide Gross by Genre')
plt.xlabel('Genres')
plt.ylabel('ROI (%)');

**Observations:**
>* **Top three genres:** Animation, Adventure, and Sci-Fi
>* **Lowest three genres:** Reality-TV, War and News
>* *Musicals are a high-risk, high-reward option*
    * Their gross can exceed Animation, or fall below the top 5 genres.

**Suggestions**
>* **Safest Genres (by Gross)** are the top three genres
    * Lowest points on the error bars indicate high gross performance even at their worst
>* Select from Action, Animation, Adventure, Fantasy, Sci-Fi, Family, or Musicals
    * All others show poor performance

## Visualizing Seasonal Performance

### Seasonal Performance - All Movies (ROI)

In [ ]:
## Creating basic overview
g = sns.barplot(data=plot_df, x='release_season_manual', y='ROI', 
                order=['Spring', 'Summer', 'Fall', 'Winter'], 
                estimator=np.mean)
g.set_xlabel('Seasons')
g.set_ylabel('ROI(%)')
g.set_title('Seasonal Performance (All Movies)');

### Seasonal Performance - All Movies (Ww Gross)

In [ ]:
## Creating basic overview
g = sns.barplot(data=plot_df, x='release_season_manual', y='worldwide_gross', 
                order=['Spring', 'Summer', 'Fall', 'Winter'], 
                estimator=np.mean)
g.set_xlabel('Seasons')
g.set_ylabel('Worldwide Gross')
g.set_title('Seasonal Performance (All Movies)');

**Observations:**
>* Summer is the best season for releases, with winter being the worst time.
>* Summer and Spring seasons seem to be the most productive seasons.
>* Fall and Winter perform worse.

**Suggestions**
>* Focus release times in Summer/Spring
>* Avoid Fall/Winter

### Genre Performance by Season

In [ ]:
## Visualizing each genre's performance by season
g = sns.catplot(data=plot_df, col='release_season_manual',
            y='ROI', kind='bar', x='genres_str', col_wrap=2, 
            aspect=1.75, order=mean_roi)
(g.set_axis_labels('Category', 'ROI (%)')
 .set_xticklabels(rotation=45)
 .set_titles("{col_name}"))
 
plt.tight_layout();

In [ ]:
## Visualizing each genre's performance by season
g = sns.catplot(data=plot_df, col='release_season_manual',
            y='worldwide_gross', kind='bar', x='genres_str', col_wrap=2, 
            aspect=1.75)#, sharex=False)
(g.set_axis_labels('Category', 'Worldwide Gross ($)')
 .set_xticklabels(rotation=45)
 .set_titles("{col_name}"))
 
plt.tight_layout();

**Observations:**
>* Springtime releases show highest gross performance on average
>* Wintertime shows lowest performances across all genres
>* The results match up with our overall view for all genres

**Suggestions**
>* To maximize profitability of musicals, release in spring
>* Avoid releasing Animations in the winter - all other seasons perform better
>* Avoid releasing news-related movies in the Spring

## Seasonal Performance -  Insights

**Observations:**
>
>The top five genres tend to perform relatively well regardless of the season with little difference between each season.
>
> Musicals show a strongest performance in the springtime - it is only worthwhile to release a musical in the spring.
>
**Questions**
> 
>What is the profitability and return on investment for each genre?

## Quarterly Performance

How would the data look when comparing seasons to quarters?

In [ ]:
## Sorting by release quarters for graphing
df_filled.sort_values('release_quarter', inplace=True)
df_filled.reset_index(drop=True, inplace=True)
# df_filled

In [ ]:
## Creating basic overview
g = sns.barplot(data=plot_df, x='release_quarter', y='worldwide_gross')
g.set_xlabel('Quarters')
g.set_ylabel('Worldwide Gross')
g.set_title('Quarterly Performance (All Movies)');

**Observations:**
>* 
>* 
>* 

**Suggestions**
>* 
>* 

In [ ]:
## Generating figure for quarterly performance breakdown
g = sns.catplot(data=plot_df, col='release_quarter',
            y='worldwide_gross', kind='bar', x='genres_str', col_wrap=2, 
            aspect=1.75)#, sharex=False)
(g.set_axis_labels('Category', 'Worldwide Gross ($)')
 .set_xticklabels(rotation=45)
 .set_titles("Quarter {col_name}"))
 
plt.tight_layout();

**Observations:**
>* 
>* 
>* 

**Suggestions**
>* 
>* 

In [ ]:
# g = sns.catplot(x = 'genres_list', y='worldwide_gross', 
#                hue = 'release_quarter',data=plot_df, kind='bar', aspect=3.65)
# g.set_xticklabels(rotation=45);

In [ ]:
# g = sns.catplot(x = 'genres_list', y='worldwide_gross',
#                 hue = 'release_season_manual',data=plot_df, kind='bar',
#                 aspect=3.65)
# g.set_xticklabels(rotation=45);

## **Comparing S & Q**

In [ ]:
## Comparing Seasonal/Quarterly breakdowns

fig, axes = plt.subplots(nrows=2, figsize=(7,7))
sns.barplot(x = 'release_season_manual', y='worldwide_gross',data=plot_df, 
            ax=axes[0], order=['Spring', 'Summer', 'Fall', 'Winter'])
sns.barplot(x = 'release_quarter', y='worldwide_gross', data=plot_df,
            ax=axes[1])

## Changing settings
axes[0].set_xlabel("Seasons")
axes[0].set_ylabel('Worldwide Gross')

axes[1].set_xlabel("Quarters")
axes[1].set_ylabel('Worldwide Gross')

plt.tight_layout();

In [ ]:
## Creating single visualization to compare seasonal and quarterly performances

fig, axes = plt.subplots(nrows=2, figsize=(17,10))
sns.barplot(x = 'genres_str', y='worldwide_gross',
            hue = 'release_season_manual',data=plot_df, ax=axes[0])
sns.barplot(x = 'genres_str', y='worldwide_gross', hue = 'release_quarter',
            data=plot_df, ax=axes[1])

## Changing settings
axes[0].set_xticklabels(axes[0].get_xticklabels(),rotation=45, ha='right')
axes[0].set_xlabel("Genres")
axes[0].set_ylabel('Worldwide Gross')
axes[0].legend(title='Seasons')

axes[1].set_xticklabels(axes[1].get_xticklabels(),rotation=45, ha='right')
axes[1].set_xlabel("Genres")
axes[1].set_ylabel('Worldwide Gross')
axes[1].legend(title='Quarters')

plt.tight_layout();

**Observations:**
>* 
>* 
>* 

**Suggestions**
>* 
>* 

# ✨ TODO

* Change to "quarterly"
* Change metric from gross to ROI
    * Profit = Gross - Budget
    * ROI = Profit / Cost
* Use ROI alongside gross
* Move towards linreg (feature selection)
* Logreg: success being ROI over certain %

# Correlations/Multicollinearity

# Statistical Testing

# Inferential Modeling

# Predictive Modeling

# Final Results